In [20]:
# import libraries
import os
import json
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
sns.set_theme(style="whitegrid")

In [3]:
# import data
df = pd.read_csv('data/tracks.csv', index_col=False)
df.drop(columns={'Unnamed: 0'}, inplace=True)

In [4]:
del df["track_id"]; del df["artist_name"]; del df["track_name"]

In [13]:
df

,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,90,rap,0.486,0.545,11.0,-7.924,1.0,0.0336,0.0652,0.004740,0.0642,0.0385,150.187,123891.0,4.0
1,91,rap,0.817,0.440,10.0,-8.482,0.0,0.0734,0.0603,0.000001,0.3300,0.5440,142.024,272113.0,4.0
2,94,rap,0.644,0.735,8.0,-5.747,1.0,0.0391,0.0521,0.144000,0.1610,0.4180,88.980,153947.0,4.0
3,96,rap,0.715,0.620,1.0,-6.005,0.0,0.0484,0.4170,0.000000,0.0822,0.1720,97.950,221520.0,4.0
4,92,rap,0.558,0.559,6.0,-9.222,1.0,0.0959,0.3710,0.000007,0.1090,0.6200,78.558,180387.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,85,r&b,0.714,0.800,11.0,-4.808,0.0,0.0504,0.1270,0.000000,0.3590,0.5890,134.002,211560.0,4.0
1002,78,r&b,0.693,0.394,0.0,-15.882,1.0,0.0428,0.0782,0.000694,0.2470,0.7210,104.114,238747.0,4.0
1003,75,r&b,0.935,0.552,10.0,-9.373,0.0,0.3350,0.1110,0.000000,0.0952,0.6150,99.993,145627.0,4.0
1004,77,r&b,0.546,0.787,0.0,-4.023,0.0,0.2120,0.3960,0.000000,0.0619,0.4970,100.559,264473.0,3.0


In [31]:
df.groupby("genre").count()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
genre,,,,,,,,,,,,,,
blues,100,100,100,100,100,100,100,100,100,100,100,100,100,100
classical,101,101,101,101,101,101,101,101,101,101,101,101,101,101
electronic,100,100,100,100,100,100,100,100,100,100,100,100,100,100
funk,101,101,101,101,101,101,101,101,101,101,101,101,101,101
jazz,102,102,102,102,102,102,102,102,102,102,102,102,102,102
metal,100,100,100,100,100,100,100,100,100,100,100,100,100,100
r&b,100,100,100,100,100,100,100,100,100,100,100,100,100,100
rap,100,100,100,100,100,100,100,100,100,100,100,100,100,100
rock,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

y_tmp = pd.DataFrame(df["genre"])
X = df.drop("genre", axis=1)
encoder = OneHotEncoder(sparse=False)
encoder.fit(y_tmp)
cols = [x.replace("x0_", "") for x in encoder.get_feature_names()]
display(len(cols))
y = pd.DataFrame(encoder.transform(y_tmp), columns=cols)
X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)# , stratify=y > 0

X:\Games\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


10

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

def scaler_pipeline():
    return make_pipeline(SimpleImputer(strategy='constant', fill_value=0), RobustScaler())

def scaler_pipeline_pwr():
    return make_pipeline(SimpleImputer(strategy='constant', fill_value=0), RobustScaler(), PowerTransformer(method='yeo-johnson', standardize=True))

ct = ColumnTransformer([
        ('stdscaled', StandardScaler(), ["popularity","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]),
       ], remainder='passthrough')

X_train_scaled = ct.fit_transform(X_train_org)
X_test_scaled  = ct.transform(X_test_org)
feature_names = ct.get_feature_names_out()
X_train = X_train_scaled
X_test  = X_test_scaled

In [8]:
feature_names

array(['stdscaled__popularity', 'stdscaled__danceability',
       'stdscaled__energy', 'stdscaled__key', 'stdscaled__loudness',
       'stdscaled__mode', 'stdscaled__speechiness',
       'stdscaled__acousticness', 'stdscaled__instrumentalness',
       'stdscaled__liveness', 'stdscaled__valence', 'stdscaled__tempo',
       'stdscaled__duration_ms', 'stdscaled__time_signature'],
      dtype=object)

In [16]:
y_train_rap = y_train.rap
y_train.columns

Index(['blues', 'classical', 'electronic', 'funk', 'jazz', 'metal', 'r&b',
       'rap', 'rock', 'techno'],
      dtype='object')

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [29]:
classifiers = [
    ('LogisticRegression', LogisticRegression(random_state=42)),
    ('KNeighborsClassifier', KNeighborsClassifier()),
    ('DecisionTreeClassifier', DecisionTreeClassifier(random_state=42)),
    ('RandomForestClassifier', RandomForestClassifier(random_state=42)),
    ('SVC', SVC(random_state=42))
]

result = {}
print("predicitng Rap!")
for gen in y_train.columns:
    for name, classifier in classifiers:
        y_target = y_train[gen]
        scores = [round(x,2) for x in cross_val_score(classifier, X_train, y_train[gen], cv=3, scoring='accuracy')]
        _tmp = {"min":min(scores), "max":max(scores), "mean":round(np.mean(scores),2)}
        print(f"cross_val_score: [{gen}]->{name}: {_tmp}")
        classifier.fit(X_train, y_target)
    print()
    for name, classifier in classifiers:
        y_pred = classifier.predict(X_test)
        _score = accuracy_score(y_test[gen], y_pred)
        print(f"test-df: [{gen}]->{name}: {round(_score, 2)}")
    print()

predicitng Rap!
cross_val_score: [blues]->LogisticRegression: {'min': 0.9, 'max': 0.91, 'mean': 0.9}
cross_val_score: [blues]->KNeighborsClassifier: {'min': 0.9, 'max': 0.9, 'mean': 0.9}
cross_val_score: [blues]->DecisionTreeClassifier: {'min': 0.9, 'max': 0.91, 'mean': 0.9}
cross_val_score: [blues]->RandomForestClassifier: {'min': 0.9, 'max': 0.91, 'mean': 0.9}
cross_val_score: [blues]->SVC: {'min': 0.9, 'max': 0.91, 'mean': 0.91}

test-df: [blues]->LogisticRegression: 0.89
test-df: [blues]->KNeighborsClassifier: 0.91
test-df: [blues]->DecisionTreeClassifier: 0.91
test-df: [blues]->RandomForestClassifier: 0.9
test-df: [blues]->SVC: 0.88

cross_val_score: [classical]->LogisticRegression: {'min': 0.99, 'max': 1.0, 'mean': 0.99}
cross_val_score: [classical]->KNeighborsClassifier: {'min': 0.99, 'max': 1.0, 'mean': 0.99}
cross_val_score: [classical]->DecisionTreeClassifier: {'min': 0.96, 'max': 0.98, 'mean': 0.97}
cross_val_score: [classical]->RandomForestClassifier: {'min': 0.98, 'max': 0

In [25]:
accuracy_score

<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>